<a href="https://colab.research.google.com/github/GuptaNavdeep1983/CS688/blob/main/TopicModelling2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [131]:
pip install python-rake

In [132]:
import io
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd 
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras import utils
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from gensim.models import Word2Vec
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import numpy as np
from collections import Counter
from pprint import pprint
import RAKE
import operator 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df = pd.read_csv("pubmed_results.csv")
df.dropna(inplace=True)
df

In [125]:
!git clone https://github.com/zelandiya/RAKE-tutorial RAKE


Cloning into 'RAKE'...
remote: Enumerating objects: 303, done.
remote: Total 303 (delta 0), reused 0 (delta 0), pack-reused 303
Receiving objects: 100% (303/303), 4.31 MiB | 19.78 MiB/s, done.
Resolving deltas: 100% (50/50), done.


In [40]:
def extract_words(text):
    temp = text.split() # Split the text on whitespace
    text_words = []

    for word in temp:
        # Remove any punctuation characters present in the beginning of the word
        while word[0] in string.punctuation:
            word = word[1:]

        # Remove any punctuation characters present in the end of the word
        while word[-1] in string.punctuation:
            word = word[:-1]

        # Append this word into our list of words.
        text_words.append(word.lower())
        
    return text_words

In [52]:
all_documents = []
all_sentences = df['title'].to_numpy()

# for line in all_sentences:
#   all_words.extend(line.strip().split())
all_documents = [nltk.word_tokenize(sent) for sent in all_sentences]

In [54]:
from nltk.corpus import stopwords
for i in range(len(all_words)):
    all_documents[i] = [w for w in all_documents[i] if w not in stopwords.words('english')]

In [55]:
def tf_idf(corpus_words):
    IDF = inv_doc_freq(corpus_words)
    
    TFIDF = []
    
    for document in corpus_words:
        TFIDF.append(Counter(document))
    
    for document in TFIDF:
        for word in document:
            document[word] = document[word]*IDF[word]
            
    return TFIDF

def inv_doc_freq(corpus_words):
    number_docs = len(corpus_words)
    
    document_count = {}

    for document in corpus_words:
        word_set = set(document)

        for word in word_set:
            document_count[word] = document_count.get(word, 0) + 1
    
    IDF = {}
    
    for word in document_count:
        IDF[word] = np.log(number_docs/document_count[word])
        
    
    return IDF

def term_document_matrix(TFIDF, word_list, word_dict):
    vocabulary_size = len(word_dict)
    number_documents = len(TFIDF)
    
    TDM = np.zeros((vocabulary_size, number_documents))
    
    for doc in range(number_documents):
        document = TFIDF[doc]
        
        for word in document.keys():
            pos = word_dict[word]
            
            TDM[pos, doc] = document[word]
            
    return TDM

def build_vocabulary(TFIDF):
    words = set()
    
    for document in TFIDF:
        words |= document.keys()
    
    word_list = list(words)
    word_dict = dict(zip(word_list, range(len(word_list))))
    
    return word_dict, word_list

In [ ]:
IDF = inv_doc_freq(all_words)

pprint(IDF)

In [ ]:
IDF


In [57]:
TFIDF = tf_idf(all_words)

In [58]:
word_dict, word_list = build_vocabulary(TFIDF)

In [59]:
TDM = term_document_matrix(TFIDF, word_list, word_dict)

In [83]:
TDM

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [73]:
def find_related_docs(tweet, TDM):
    new_vector = np.zeros(TDM.shape[1])
    
    for word in tweet:
        if word in word_dict:
          pos = word_dict[word]
          new_vector += TDM[pos, :]
        
    # Now the entries of new_vector tell us which documents are activated by this one.
    # Let's extract the list of documents sorted by activation
    doc_list = sorted(zip(range(TDM.shape[1]), new_vector), key=lambda x:x[1], reverse=True)
    
    return doc_list

In [74]:
research_words = ["Obesity", "Cancer", "Covid-19", "wearable", "mental health", "influenza"]
related = find_related_docs(research_words, TDM)

In [ ]:
for tweet, score in related[:100]:
    print(tweet, " ".join(all_documents[tweet]))

In [138]:

 
rake_object = RAKE.rake('SmartStoplist.txt', 5, 3, 4)

TypeError: ignored